In [ ]:
import matplotlib.pyplot as pp
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tp

In [ ]:
url = 'https://raw.githubusercontent.com/mattiasvillani/AdvBayesLearnCourse/master/Labs/LidarData.dat'
data = pd.read_table(url, sep='\s+')
x = data['Distance'].values.reshape((-1, 1))
y = data['LogRatio'].values

In [ ]:
pp.scatter(x, y)

In [ ]:
def build_gaussian_process(amplitude, length_scale, noise_variance):
    return tp.distributions.GaussianProcess(
        kernel=tp.math.psd_kernels.ExponentiatedQuadratic(amplitude, length_scale),
        observation_noise_variance=noise_variance,
        index_points=x,
    )

model = tp.distributions.JointDistributionNamed(dict(
    amplitude=tp.distributions.LogNormal(loc=0., scale=np.float64(1.)),
    length_scale=tp.distributions.LogNormal(loc=0., scale=np.float64(1.)),
    noise_variance=tp.distributions.LogNormal(loc=0., scale=np.float64(1.)),
    observations=build_gaussian_process,
))

In [ ]:
x_ = model.sample()
model.log_prob(x_)

In [ ]:
@tf.function(autograph=False, experimental_compile=False)
def log_probability(amplitude, length_scale, noise_variance):
    return model.log_prob(dict(
        amplitude=amplitude,
        length_scale=length_scale,
        noise_variance=noise_variance,
        observations=y,
    ))

num_draws = 100
num_burnin_steps = 50

step_size = 0.1
num_leapfrog_steps = 8

num_adaptation_steps = int(0.8 * num_burnin_steps)
acceptance_probability = 0.75

constrain_positive = tp.bijectors.Shift(np.finfo(np.float64).tiny)(tp.bijectors.Exp())

sampler = tp.mcmc.TransformedTransitionKernel(
    tp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=log_probability,
        step_size=tf.cast(step_size, tf.float64),
        num_leapfrog_steps=num_leapfrog_steps,
    ),
    bijector=[
        constrain_positive,
        constrain_positive,
        constrain_positive,
    ],
)
sampler = tp.mcmc.DualAveragingStepSizeAdaptation(
    inner_kernel=sampler,
    num_adaptation_steps=num_adaptation_steps,
    target_accept_prob=tf.cast(acceptance_probability, tf.float64),
)

initial_state = [tf.cast(x, tf.float64) for x in [1., 1., 1.]]

@tf.function(autograph=False, experimental_compile=False)
def sample():
    return tp.mcmc.sample_chain(
        kernel=sampler,
        current_state=initial_state,
        num_results=num_draws,
        num_burnin_steps=num_burnin_steps,
        trace_fn=lambda current_state, kernel_results: kernel_results,
    )

samples, results = sample()